# A Minimalist Computation Graph Framework

In [14]:
import numpy as np
from IPython.display import Image

https://github.com/davidrosenberg/mlcourse/blob/gh-pages/Notebooks/computation-graph/computation-graph-framework.ipynb

# Value Nodes

In [15]:

class ValueNode(object):
    """Computation graph node having no input but simply holding a value"""
    def __init__(self, node_name):
        self.node_name = node_name
        self.out = None
        self.d_out = None

    def forward(self):
        self.d_out = np.zeros(self.out.shape)
        return self.out

    def backward(self):
        return self.d_out

    def get_predecessors(self):
        return []

To give a ValueNode a particular output value, we directly set it. It should always be a numpy array. For example, for a scalar value we can set it as follows:

In [16]:
x = ValueNode("x")
x.out = np.array(3)
x.d_out = np.array(1)
print(x.backward())

1


## Square Node

In [17]:
class SquareNode(object):
    """Node for squaring a scalar"""
    def __init__(self, x, node_name):
        """ 
        Parameters:
        x: a node producing a scalar as a numpy array
        node_name: node's name (a string)
        """
        self.node_name = node_name
        self.out = None
        self.d_out = None
        self.x = x
    def forward(self):
        self.out = self.x.out**2
        self.d_out = np.zeros(self.out.shape)
        return self.out
    def backward(self):
        # Preconditions: self.d_out contains the partial derivatives of the graph output w.r.t. self.out
        d_x = self.d_out*2*self.x.out
        self.x.d_out += d_x
        return self.d_out

    def get_predecessors(self):
        """Get list of node's predecessors"""
        return [self.x]

In [18]:
x = ValueNode("x")
f = SquareNode(x, "square_node")
# set input value
x.out = np.array(7)
# forward pass
x.forward()
f.forward()

49

In [19]:
f.d_out = np.array(1)
f.backward()
x.backward()

array(14.)

## Gradient descent with a computation graph

In [20]:
x = ValueNode("x")
f = SquareNode(x, "square_node")

obj_vals = []
x_vals = []
num_steps = 10
step_size = .3
x.out = np.array(3)
for step in range(num_steps):
    x_vals.append(x.out)
    x.forward()
    J = f.forward()
    print("x=",x.out,"J(x)=",J)
    obj_vals.append(J)
    f.d_out = 1
    f.backward()
    x_grad = x.backward()
    x.out = x.out - step_size*x_grad
    

x= 3 J(x)= 9
x= 1.2000000000000002 J(x)= 1.4400000000000004
x= 0.4800000000000001 J(x)= 0.23040000000000008
x= 0.19200000000000006 J(x)= 0.03686400000000002
x= 0.07680000000000003 J(x)= 0.005898240000000005
x= 0.030720000000000018 J(x)= 0.0009437184000000011
x= 0.012288000000000007 J(x)= 0.00015099494400000018
x= 0.004915200000000003 J(x)= 2.4159191040000033e-05
x= 0.0019660800000000016 J(x)= 3.865470566400006e-06
x= 0.0007864320000000008 J(x)= 6.184752906240012e-07


## A Computation Graph with Shared Input

In [21]:
class ExpNode(object):
    """Node for computing e^x for a scalar x"""
    def __init__(self, x, node_name):
        """ 
        Parameters:
        x: a node producing a scalar
        node_name: node's name (a string)
        """
        self.node_name = node_name
        self.out = None
        self.d_out = None
        self.x = x

    def forward(self):
        self.out = np.exp(self.x.out)
        self.d_out = np.zeros(self.out.shape)      
        return self.out

    def backward(self):
        # Preconditions: 
        #   self.out contains the output value of the node, namely e^(x.out)
        #   self.d_out contains the partial derivatives of the graph output w.r.t. self.out
        d_x = self.d_out*self.out
        self.x.d_out += d_x 
        return self.d_out
    
    def get_predecessors(self):
        return [self.x]

In [22]:
class ProductNode(object):
    """Node for computing the product of two scalars"""
    def __init__(self, x, y, node_name):
        self.node_name = node_name
        self.out = None
        self.d_out = None
        self.x = x # Store a reference to node providing input to this node
        self.y = y

    def forward(self):
        self.out = self.x.out*self.y.out
        self.d_out = np.zeros(self.out.shape)
        return  self.out

    def backward(self):
        d_x = self.d_out*self.y.out
        d_y = self.d_out*self.x.out
        self.x.d_out += d_x
        self.y.d_out += d_y
        #print(d_x)
        return self.d_out

    def get_predecessors(self):
        return [self.x, self.y]

In [23]:
x = ValueNode("x")
f = SquareNode(x, "square node")
g = ExpNode(x, "exponentiation node") 
h = ProductNode(f, g, "product node")

x.out = np.array(7)
x.forward()
f.forward()
g.forward()
h.forward() 
print("x = ",x.out)
print("f(x) = x^2 = ",f.out)
print("g(x) = e^x = ",g.out)
print("J=h(f(x),g(x)) = x^2 e^x = ",h.out)

x =  7
f(x) = x^2 =  49
g(x) = e^x =  1096.6331584284585
J=h(f(x),g(x)) = x^2 e^x =  53735.024762994464


In [24]:
h.d_out = np.array(1)
h.backward()
f_x = f.out
g_x = g.out
J = h.out
dJdf = f.d_out
print("At x=",x.out," we have f(x)=",f_x,", g(x)=",g.out,", h(x)=",J)
print("At those values, we have dJ/d[f(x)]=", dJdf)

At x= 7  we have f(x)= 49 , g(x)= 1096.6331584284585 , h(x)= 53735.024762994464
At those values, we have dJ/d[f(x)]= 1096.6331584284585


## checking gradient

In [25]:

delta = np.random.randn()/100000  #choose a random perturbation (we can rerun this several times)
J_first_order_change = np.dot(delta, dJdf) #predicted change using derivative
f.out = f_x + delta # change f.out by delta
h.forward() # compute J_new=h(f(x)+delta, g(x))
J_new = h.out
J_actual_change = J_new - J
print("For delta=",delta,", J(f(x)+delta, g(x)) - J(f(x),g(x)) = ", J_new - J)
print("First order prediction of J(f(x)+delta, g(x)) - J(f(x),g(x)) = ", J_first_order_change)

For delta= 8.533600952616852e-06 , J(f(x)+delta, g(x)) - J(f(x),g(x)) =  0.00935822977044154
First order prediction of J(f(x)+delta, g(x)) - J(f(x),g(x)) =  0.009358229765436321


## backpro

In [26]:
x.out = np.array(7) #Set the value of the input node
# Forward pass -- call forward() on nodes in topological order (parents before children)
x.forward()
f.forward()  
g.forward()
h.forward()
# Backward pass -- call backward() on nodes in reverse topological order (children before parents)
h.d_out = np.array(1) #initialize backpropagation
h.backward() 
f.backward()
g.backward()

array(49.)

## Automating the forward and backward passes

In [27]:
def sort_topological(sink):
    """Returns a list of the sink node and all its ancestors in topologically sorted order.
    Subgraph of these nodes must form a DAG.  understand the function suspend ! """
    L = []
    T = set()
    P = set()

    def visit(node):
        if node in P:
            return
        if node in T:
            raise "your graph is not a DAG!"
        T.add(node)
        for predecessor in node.get_predecessors():
            visit(predecessor)
        P.add(node)
        L.append(node)
    visit(sink)
    return L

In [28]:

for node in sort_topological(h):
    print(node.node_name,":  ",node)

x :   <__main__.ValueNode object at 0x0000018A459D1550>
square node :   <__main__.SquareNode object at 0x0000018A459D1240>
exponentiation node :   <__main__.ExpNode object at 0x0000018A45624D30>
product node :   <__main__.ProductNode object at 0x0000018A45624E80>


In [30]:
def forward_graph(graph_output_node):
    node_list = sort_topological(graph_output_node)
    for node in node_list:
        print("Running forward on node ",node.node_name)
        out = node.forward()
    return out

In [29]:

def backward_graph(graph_output_node):
    """
    Assumes that forward_graph has already been called on graph_output_node.
    Sets d_out of each node to the appropriate derivative.
    """ 
    nodes_list = sort_topological(graph_output_node)
    nodes_list.reverse()
        
    graph_output_node.d_out = np.array(1) # Derivative of graph output w.r.t. itself is 1

    for node in nodes_list:
        print("Running backward on node ",node.node_name)
        node.backward()

In [31]:

x.out = np.array(2.) # set input value
out = forward_graph(h) # run forward pass for graph output node h
print("J(x)=x^2 e^x=",out," for x=",x.out)
backward_graph(h) # run backward pass
print("J'(x)=",x.d_out," for x=",x.out)

Running forward on node  x
Running forward on node  square node
Running forward on node  exponentiation node
Running forward on node  product node
J(x)=x^2 e^x= 29.5562243957226  for x= 2.0
Running backward on node  product node
Running backward on node  exponentiation node
Running backward on node  square node
Running backward on node  x
J'(x)= 59.1124487914452  for x= 2.0
